In [ ]:
import torch
from torch import nn
from torch.nn import functional
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
input_size = 19200  # 120x160
num_classes = 3
batch_size = 2
learning_rate = 0.0001

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Resize(120),
    transforms.Grayscale(1)]
)

In [ ]:
train_dataset = torchvision.datasets.ImageFolder('./rock_paper_scissors_data/train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = torchvision.datasets.ImageFolder('./rock_paper_scissors_data/test', transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)

In [ ]:
classes = ['paper', 'rock', 'scissors']

In [ ]:
class FeedForwardNeuralNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(FeedForwardNeuralNet, self).__init__()
        self.linear1 = nn.Linear(input_size, 3000)
        self.linear2 = nn.Linear(3000, 3000)
        self.linear3 = nn.Linear(3000, num_classes)
        self.Relu = nn.ReLU()
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.Relu(x)
        x = self.linear2(x)
        x = self.Relu(x)
        x = self.linear3(x)
        # no activation or softmax used 
        return x

In [ ]:
model = FeedForwardNeuralNet(input_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def test_model(model, test_loader):
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for images, labels in test_loader:
            images = images.reshape(-1, input_size).to(device)
            labels = labels.to(device)
            results = model(images)
            # max returns (value ,index)
            _, predicted = torch.max(results.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

        acc = n_correct / n_samples
        print(f'Accuracy of the model: {acc * 100.0:.4f} %')
        return acc

In [ ]:
def train_model(train_loader, num_epochs, model):
    total_steps = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            # flatten images
            images = images.reshape(-1, input_size).to(device)
            labels = labels.to(device)

            # forward pass
            results = model(images)
            loss = criterion(results, labels)

            # backwards and optimise
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (i+1) % 25 == 0:
                print(f'Epoch: {epoch+1} out of {num_epochs}, Step: {i+1} out of {total_steps}, Loss: {loss.item():.4f}') 
        test_model(model, test_loader)    

In [ ]:
train_model(train_loader, 30, model)